In [1]:
import traceback
from getpass import win_getpass

import numpy as np
import nlopt
import math
import random
from scipy.linalg import block_diag, expm
from numpy import *

In [2]:
def generate_omega(n):
    """Generates an Omega matrix of size n"""
    omega = np.zeros((n,n))

    assert 2*int(n/2)==n, f"Omega generation expected even number, got {n}"

    for i in range(int(n/2)):
        omega[2*i,2*i+1]=1
        omega[2*i+1,2*i]=-1

    return omega

In [33]:
def symplectic_trace1(Z):
    """Return the symplectic eigenvalues of Z
    Parameters:
        Z a numpy matrix
    Returns:
        sum of symplectic eigenvalues"""

    omega = generate_omega(Z.shape[0])
    eigenvalues, eigenvectors = np.linalg.eig(1j*(omega@Z))
    sum = 0
    for i in range(len(eigenvalues)):
        assert np.imag(eigenvalues[i]) < 10**(-15), f"Symplectic eigenvalue imaginary part no negligible, got {eigenvalues[i]}"

    for i in range(int(len(eigenvalues)/2)):
        sum = sum + np.real(eigenvalues[2*i])
    return sum

In [34]:
def symplectic_trace2(Z):
    """Return the symplectic eigenvalues of Z
    Parameters:
        Z a numpy matrix
    Returns:
        sum of symplectic eigenvalues"""

    omega = generate_omega(Z.shape[0])
    eigenvalues, eigenvectors = np.linalg.eig(1j*(omega@Z))
    sum1 = 0
    sum2 = 0

    for i in range(int(len(eigenvalues)/2)):
        sum1 = sum1 + np.real(eigenvalues[2*i])
        sum2 = sum2 + np.absolute(eigenvalues[2*i])
    return sum1, sum2

In [37]:
a=4
b=6
c=2
d=3
Z = np.matrix([[-a,0,c,0],
               [0,-a,0,d],
               [c,0,b,0],
               [0,d,0,b]])

print(symplectic_trace1(Z))
print(symplectic_trace2(Z))



11.171156458231291
(np.float64(11.171156458231291), np.float64(11.171156458231291))


In [4]:
def measurement(theta, psi, phi):
    """Return the measurement for specific angles

    Parameters:
        theta: theta angle in [0,pi]
        psi: psi angle in [0,pi]
        z: phi angle in [0,2pi]
    Returns:
        A 4x4 numpy array
    """
    u=np.matrix([math.cos(theta)*math.cos(psi-phi), math.cos(theta)*math.sin(psi-phi), math.sin(theta)*math.cos(psi), math.sin(theta)*math.sin(psi)])
    v=np.matrix([math.cos(theta)*math.cos(psi-phi),math.cos(theta)*math.sin(psi-phi),math.sin(theta)*math.cos(psi),math.sin(theta)*math.sin(psi)])
    return np.outer(u,v.T)

In [5]:
def generate_random_haar(n):
    """Generates a Haar-random unitary matrix of size n"""
    F = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    Q, R = np.linalg.qr(F)
    return Q @ np.diag(np.exp(1j * np.random.rand(n) * 2 * np.pi))

def generate_symplectic_orthogonal(n):
    """Generates a symplectic orthogonal matrix of size n"""
    B = generate_random_haar(n)
    Re = np.real(B)
    Im = np.imag(B)
    return np.block([[Re, -Im],
                    [Im,  Re]])

In [6]:
def generate_random_symplectic(n, c):
    """Generate random symplectic matrix with specified symplectic eigenvalues.
    Parameters:
        n: dimension (2nx2n)
        c: symplectic eigenvalue(s) - either a scalar or n-vector
    Returns:
        A symplectic matrix
    """
    assert (len(c)==1 or len(c)==n),  (f"Expected vector component length for "
                                       f"symplectic matrix generation either 1 or {n}, got: {len(c)}")
    for scalar in c:
        assert scalar >= 1, f"Expected vector component for symplectic matrix generation >=, got: {scalar}"

    s = np.zeros(2*n)
    if(len(c)==1):
        s[0]=np.sqrt(c[0])
        s[1:n] = np.sort(1 + (s[0] - 1) * np.random.rand(n-1))[::-1]
        s[n:] = 1.0 / s[:n]
    else:
        c = np.sort(c)
        s[0:n] = c[::-1]
        s[n:2*n] = 1.0 / s[0:n]

    U=generate_symplectic_orthogonal(n)
    V=generate_symplectic_orthogonal(n)

    return U @ np.diag(s) @ V

In [7]:
def find_random_entangled_state(entg = 1):
    """Searches for a random state with a specific entanglement level
    Parameters:
        entg: entanglement level
    Returns:
        A 4x4 matrix
    """
    int = 5
    rot = 10
    num = 300000

    P = np.matrix([[1,0,0,0],
                  [0,0,1,0],
                  [0,1,0,0],
                  [0,0,0,1]])

    x = 1
    y = 1.1

    a=1
    b=10

    for i in range(num):
        # generate thermal state CM
        g1 = np.matrix([[random.uniform(x,y),0],
                        [0,random.uniform(x,y)]]),
        g2 = block_diag(g1, g1)


        # generate random symplectic transformations with symplectic eigenvalues between 1 and 7.5
        S=generate_random_symplectic(2, (random.uniform(a, b), random.uniform(a, b)))
        # apply symplectic transformations to obtain a general CM

        g3 = S.transpose()@g2@S
        cm = P.transpose()@g3@P

        # calculate the logarithmic negativity
        det_11 = np.linalg.det(cm[0:2, 0:2])
        det_22 = np.linalg.det(cm[2:4, 2:4])
        det_12 = np.linalg.det(cm[0:2, 2:4])
        det_cm = np.linalg.det(cm)

        f = (0.5 * (det_11 + det_22) - det_12
             - np.sqrt((0.5 * (det_11 + det_22) - det_12)**2 - det_cm))

        EN = -0.5 * np.log2(f)
        EN1 = np.round(EN * rot) / rot

        if EN1 == entg/int:
            return cm
    return None

In [8]:
def check_constraints(c_opt, P_list, min_val, N_measurements, verbose = False):
    """
    Sanity check: verify all constraints are satisfied

    Returns:
        dict with constraint values and whether they're satisfied
    """
    W = np.sum([c_opt[k] * P_list[k] for k in range(N_measurements)], axis=0)

    # Check 1: W ≥ 0 (PSD)
    eigvals = np.linalg.eigvalsh(W)
    min_eigval = np.min(np.real(eigvals))
    W_psd = min_eigval >= -1e-6  # Small tolerance for numerical errors

    # Check 2: sTr(Z11) + sTr(Z22) ≥ 0.5
    Z11 = W[:W.shape[0]//2, :W.shape[0]//2]
    Z22 = W[W.shape[0]//2:, W.shape[0]//2:]

    sTr_Z11 = symplectic_trace(Z11)
    sTr_Z22 = symplectic_trace(Z22)


    sTr_sum = sTr_Z11 + sTr_Z22
    sTr_satisfied = sTr_sum >= 0.5

    # Check 3 & 4: 0 ≤ c * m ≤ 1
    min_satisfied = final_value >= 0
    max_satisfied = final_value <= 1

    results = {
        'min_eigval_W': min_eigval,
        'W_is_PSD': W_psd,
        'sTr_sum': sTr_sum,
        'sTr_satisfied': sTr_satisfied,
        'min_satisfied': min_satisfied,
        'max_satisfied': max_satisfied,
        'objective_value': min_val,
        'all_constraints_ok': W_psd and sTr_satisfied
    }

    if verbose:
        print("=== Constraint Check ===")
        print(f"Objective (w·m): {min_val:.6f}")
        print(f"Min eigenvalue of W: {min_eigval:.6f} {'✓' if W_psd else '✗'}")
        print(f"det(Z11): {det_Z11:.6f}")
        print(f"det(Z22): {det_Z22:.6f}")
        if sTr_sum is not None:
            print(f"sTr(Z11) + sTr(Z22): {sTr_sum:.6f} {'✓' if sTr_satisfied else '✗'} (need ≥ 0.5)")
        else:
            print(f"sTr constraint: ✗ (negative determinants)")
        print(f"All constraints satisfied: {'YES ✓' if results['all_constraints_ok'] else 'NO ✗'}")

    return results

In [9]:

def steering_detection(P_list, m_list, n_vars):
    """
    Detect steering using NLopt with specific constraints

    Constraints:
    - W ≥ 0 (PSD)
    - sTr(Z11) + sTr(Z22) ≥ 0.5
    """

    # Objective function
    def objective(c, grad):
        obj = np.dot(c, m_list)
        if grad.size > 0:
            grad[:] = m_list
        return float(obj)

    # Constraint 1: W ≥ 0 (PSD)
    def constraint_W_psd(c, grad):
        W = np.sum([c[k] * P_list[k] for k in range(n_vars)], axis=0)
        eigvals = np.linalg.eigvalsh(W)
        min_eigval = np.min(np.real(eigvals))

        if grad.size > 0:
            grad[:] = 0
        return float(min_eigval)

    # Constraint 2: sTr(Z11) + sTr(Z22) ≥ 0.5 (with MIN_DET check)
    def constraint_symplectic_trace(c, grad):
        W = np.sum([c[k] * P_list[k] for k in range(n_vars)], axis=0)
        Z11 = W[:W.shape[0]//2, :W.shape[0]//2]
        Z22 = W[W.shape[0]//2:, W.shape[0]//2:]

        sTr_Z11 = symplectic_trace(Z11)
        sTr_Z22 = symplectic_trace(Z22)
        constraint_val = sTr_Z11 + sTr_Z22 - 0.5
        print(constraint_val)
        input()

        if grad.size > 0:
            grad[:] = 0
        # print(f'cond4:{constraint_val}')
        return float(constraint_val)

    def constraint_lower(c, grad):
        # c * m >= 0
        f = np.sum([c[k] * m_list[k] for k in range(n_vars)], axis=0)
        if grad.size > 0:
            grad[:] = 0
        return -f

    def constraint_upper(c, grad):
        # c * m <= 1
        f = np.sum([c[k] * m_list[k] for k in range(n_vars)], axis=0)
        if grad.size > 0:
            grad[:] = 0
        return f - 1
    # Set up optimizer
    # opt = nlopt.opt(nlopt.LN_COBYLA, n_vars)
    opt = nlopt.opt(nlopt.LD_SLSQP, n_vars)
    # opt = nlopt.opt(nlopt.LN_AUGLAG, n_vars)
    # opt.set_local_optimizer(nlopt.opt(nlopt.LN_COBYLA, n_vars))
    # opt = nlopt.opt(nlopt.GN_ISRES, n_vars)


    opt.set_min_objective(objective)

    # Add all constraints
    opt.add_inequality_constraint(constraint_W_psd, 1e-6)
    opt.add_inequality_constraint(constraint_symplectic_trace, 1e-6)
    opt.add_inequality_constraint(constraint_lower, 1e-6)
    opt.add_inequality_constraint(constraint_upper, 1e-6)

    # Set bounds
    # opt.set_lower_bounds(-10 * np.ones(n_vars))
    # opt.set_upper_bounds(10 * np.ones(n_vars))

    # Set tolerances
    opt.set_xtol_rel(1e-6)
    opt.set_maxeval(10000)

    # Initial guess
    c0 = np.random.randn(n_vars)

    # Optimize
    try:
        c_opt = opt.optimize(c0)
        final_value = opt.last_optimum_value()
        return final_value, c_opt
    except nlopt.RoundoffLimited:
        final_value = opt.last_optimum_value()
        c_opt = c0  # Return last attempt
        print(f"  Roundoff limited, returning best found: {final_value:.6f}")
        return final_value, c_opt
    except np.linalg.LinAlgError as e:
        print(f"  LinAlgError during optimization: {e}")
        return np.inf, np.zeros(n_vars)
    except Exception as e:
        print(f"  Optimization failed")
        traceback.print_exc()
        return np.inf, None


In [10]:
# Main detection loop
N_measurements = 14
num_rep = 5
N_entanglement = 1

for kdx in range(1, N_entanglement + 1):
    n = np.zeros(N_measurements, dtype=int)

    # for jdx in range(num_rep):
        # Generate state
    while(True):
        covariance_matrix = find_random_entangled_state(kdx)

        # Generate random measurements
        theta = np.pi * np.random.rand(N_measurements)
        psi = np.pi * np.random.rand(N_measurements)
        phi = 2 * np.pi * np.random.rand(N_measurements)

        P_list = []
        m_list = []

        for k in range(N_measurements):
            P_k = measurement(theta[k], psi[k], phi[k])
            P_list.append(P_k)
            m_list.append(np.real(np.trace(P_k @ covariance_matrix)))

        # Run optimization
        final_value, w_opt = steering_detection(P_list, m_list, N_measurements)
        results = check_constraints(w_opt, P_list, final_value, N_measurements, False)

        # print(f"kdx={kdx}, jdx={jdx}, min_val={min_val:.6f}")

        # Check if steering detected
        if results['all_constraints_ok']:
            print(f"  → Steering detected! ")
            results = check_constraints(w_opt, P_list, final_value, N_measurements, True)
            break
        else:
            print(f"  → Nope {final_value}")



  Optimization failed


Traceback (most recent call last):
  File "/tmp/ipykernel_553670/3069010938.py", line 86, in steering_detection
    c_opt = opt.optimize(c0)
  File "/home/andrei/.conda/envs/QuantumMeasurementSimulationOptimization/lib/python3.13/site-packages/nlopt/nlopt.py", line 452, in optimize
    return _nlopt.opt_optimize(self, *args)
           ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "/tmp/ipykernel_553670/3069010938.py", line 34, in constraint_symplectic_trace
    sTr_Z22 = symplectic_trace(Z22)
  File "/tmp/ipykernel_553670/665180060.py", line 12, in symplectic_trace
    assert np.imag(eigenvalues[i]) < 10**(-10), f"Symplectic eigenvalue imaginary part not negligible, got {eigenvalues[i]}"
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: Symplectic eigenvalue imaginary part not negligible, got 0.7258562012863714j


TypeError: 'NoneType' object is not subscriptable